In [ ]:
import pandas as pd
import json
import os
import requests
import re
import dateparser
import logging
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dotenv import load_dotenv
from time import sleep

from save_pdf import DataFrameToMarkdownPDF
from get_html import GetHtml
from grupos import Grupos
from get_url import GetOnlineUrls
from send_email import SendEmail

load_dotenv()

BASE_URL = os.getenv('BASE_URL')
DAYS = os.getenv('DAYS')  # Lê a variável DAYS do arquivo .env

try:
    DAYS = int(DAYS)  # Converte DAYS para inteiro
except ValueError:
    logging.error("DAYS must be an integer.")
    raise
    
if BASE_URL:
    url_scraper = GetOnlineUrls(BASE_URL)
    url_scraper.get_online_urls_and_save()
else:
    logging.error("BASE_URL not found in .env file")

grupos = Grupos(DAYS)
pdf_generator = DataFrameToMarkdownPDF(pdf_filename='ransomscraper.pdf')
get_html = GetHtml()
df = get_html.run()

df_disponivel = df[df['infos'] == True]
df_to_analize = df_disponivel.drop_duplicates(subset='html')
df_to_analize = df_disponivel.drop_duplicates(subset='grupo')
df_to_analize.reset_index(drop=True, inplace=True)

def processar_grupos(grupos, df_to_analize):
    resultados = []  # Lista para armazenar os resultados

    for index, row in df_to_analize.iterrows():
        try:
            grupo = row['grupo']
            html_content = row['html']
            
            # Chamar o método correspondente com base no valor da coluna 'grupo'
            if grupo == 'ransomhouse':
                result = grupos.ransomhouse(html_content)
            elif grupo == 'monti':
                result = grupos.monti(html_content)
            elif grupo == 'play':
                result = grupos.play(html_content)
            elif grupo == 'handala':
                result = grupos.handala(html_content)
            elif grupo == 'blackbyte':
                result = grupos.blackbyte(html_content)
            else:
                logging.warning(f"Grupo '{grupo}' ainda não possui um scraper específico. Link para obter o html: {row['links_online']}")
                continue  # Pular para o próximo grupo se não houver scraper

            # Adiciona os resultados à lista, incluindo o grupo como uma nova chave
            for item in result:
                item['grupo'] = grupo  # Adicionando o grupo ao resultado
                resultados.append(item)  # Adicionando cada resultado à lista

        except Exception as e:
            logging.error(f"Erro ao processar o grupo '{grupo}': {e}")

    return resultados

data = processar_grupos(grupos, df_to_analize)
pdf_generator.generate_pdf(data, df)

sleep(10)

# Defina os caminhos dos arquivos e as informações do email
pdf_file_path = os.path.join(os.getcwd(), 'ransomscraper.pdf')  # Caminho para o PDF
json_file_path = os.path.join(os.getcwd(), 'groups.json')   # Caminho para o JSON

email_sender = SendEmail()
email_subject = "Relatório e Dados Anexados"
email_body = "Segue em anexo o relatório em PDF e os dados em JSON."

# Enviar e-mail
email_sender.send_email(email_subject, email_body, pdf_file_path, json_file_path)


